# Prism bug
This notebook illustrates a bug in Prism that causes problems for my implementation.

## Example input files

In [33]:
!cat model.prism

dtmc

module cell1
    s0 : [0..1] init 0;
    c0 : [0..1] init 0;
    [] s0=0 -> 1 : (s0'=0) & (c0'=0) + 0 : (s0'=1) & (c0'=1);
    [] s0=1 -> 1 : (s0'=0) & (c0'=0) + 0 : (s0'=1) & (c0'=1);
endmodule

const double p;
const double q=1-p;
module cell2
    s1 : [0..1] init 0;
    c1 : [0..1] init 0;
    [] s1=0 -> q : (s1'=0) & (c1'=0) + p : (s1'=1) & (c1'=1);
    [] s1=1 -> 1 : (s1'=0) & (c1'=0) + 0 : (s1'=1) & (c1'=1);
endmodule

formula n = c0 + c1 + 0;

In [34]:
!cat constraints.props

P>=0.5 [ G n <= 0 ]
P>=0.5 [ G n <= 1 ]

## Inconsistent results from prism

The following three commands should yield the same result, viz. "true" for both properties, but they do not.
The parametric engine yields incorrect results.

Note that 0.00000000000000001=0 and 0.99999999999999999=1 by the precision of Prism.
If 0 and 1 are used instead, Prism does not terminate.

In [1]:
!prism model.prism constraints.props -const p=0 > output/true_const.txt
!grep "Result:" output/true_const.txt
!echo

!prism model.prism constraints.props -param p=0:0.00000000000000001 > output/true_param_1.txt
!grep "Result:" output/true_param_1.txt
!echo

!prism model.prism constraints.props -param p=0:0.000000000000000001,q=0.99999999999999999:1 > output/true_param_2.txt
!grep "Result:" output/true_param_2.txt

Result: true
Result: true

Result: ([0.0,0.0]): false
Result: ([0.0,0.0]): true

Result: ([0.0,0.0],[1.0,1.0]): false
Result: ([0.0,0.0],[1.0,1.0]): true


The commands below should all yield the same result, viz. "false" for the first property and "true" for the second, and they do. In conclusion, the option "-const" seems to be working, whereas "-param" outputs "false" too often.

In [38]:
!prism model.prism constraints.props -const p=1 > output/false_const.txt
!grep "Result:" output/false_const.txt
!echo

!prism model.prism constraints.props -param p=0.99999999999999999:1 > output/false_param_1.txt
!grep "Result:" output/false_param_1.txt
!echo

!prism model.prism constraints.props -param p=0.99999999999999999:1,q=0:0.000000000000000001 > output/false_param_2.txt
!grep "Result:" output/false_param_2.txt

Result: false
Result: true

Result: ([1.0,1.0]): false
Result: ([1.0,1.0]): true

Result: ([1.0,1.0],[0.0,0.0]): false
Result: ([1.0,1.0],[0.0,0.0]): true
